<a href="https://colab.research.google.com/github/patrickbryant1/BL8064/blob/main/BL8064.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BL8064

Hi and welcome to the protein structure prediction exercise for the course [Methods in Molecular Life Science](https://www.su.se/english/search-courses-and-programmes/bl8064-1.684212).
This exercise is about assessing the impact of mutations using protein structure prediction.
\
Did you know that proteins interact to form intricate signaling networks?
If mutations occur in the interaction interfaces - these networks can be perturbed resulting in functional changes!
\
Here we will assess the impact of mutations of **your choice** on proteins of **your choice** using protein structure prediction of interaction networks.
\
\
Please follow the instructions below.

In [ ]:
#@title Install dependencies
#@markdown Make sure your runtime is GPU.
#@markdown In the menu above do: Runtime --> Change runtime type --> Hardware accelerator (set to GPU)

#@markdown **Press play.**

#@markdown Simply press play on each cell below and follow the instructions.

#@markdown You will have to restart the runtime after this finishes to include the new packages.
#@markdown In the menu above do: Runtime --> Restart runtime \
#@markdown ***Do not restart before all packages are installed.***

#@markdown Don't worry about all the errors that pip give below, these are resolved in the end.
!pip install --upgrade "jax[cuda12_local]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install ml-collections==0.1.1
!pip install dm-haiku==0.0.11
!pip install pandas==1.3.5
!pip install biopython==1.81
!pip install chex==0.1.5
!pip install dm-tree==0.1.8
!pip install immutabledict==2.0.0
!pip install scipy==1.7.3
!pip install tensorflow==2.11.0
!pip install rdkit-pypi
!pip install py3Dmol
!pip install numpy --upgrade

In [24]:
#@title #Map your mutations to the predicted AlphaFold (AF) structure.

#@markdown #Inputs
#@markdown - *ID* - for your protein of interest (POI). This has to be the ID from the [AlphaFoldDB](https://alphafold.ebi.ac.uk/entry/Q6FXU7) (which should be the same as the Uniprot ID). For PDR1 this is Q6FXU7 (you can search the AlphaFoldDB to find this information).
#@markdown - *Mutations* - Provide a list of mutations separated by comma in the form Y270D (amino acid Y at pos 270 is mutated to D)

import sys, os
from google.colab import files
import pandas as pd
import numpy as np
import py3Dmol
import pdb


ID = "Q6FXU7" #@param {type:"string"}
Mutations = "F669L,V1048F,Y270D" #@param {type:"string"}

#Make a result directory
OUTDIR="/content/"+ID+'/'
if not os.path.exists(OUTDIR):
  os.mkdir(OUTDIR)
print('All results will be saved to', OUTDIR)
#Get the structure from the AFDB

if not os.path.exists(OUTDIR+'/AF-'+ID+'-F1-model_v4.pdb'):
  print('Downloading protein of interest')
  !wget https://alphafold.ebi.ac.uk/files/AF-$ID-F1-model_v4.pdb
  !mv /content/AF-$ID-F1-model_v4.pdb $OUTDIR
else:
  print('Protein of interest downloaded')

#Get mutation positions
MUTS = Mutations.split(',')
TARGET_POSITIONS = [int(x[1:-1]) for x in MUTS]

view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
view.addModel(open(OUTDIR+'/AF-'+ID+'-F1-model_v4.pdb','r').read(),'pdb')
view.setStyle({'chain':'A'},{'cartoon': {'color':'green'}})


print('Viewing the POI', ID, 'in cartoon style and the positions for the input mutations in magenta spheres.')
print('The structure is coloured by plDDT (confindence - see next section):')
print('Orange: Very low (pLDDT < 50)')
print('Yellow: Low (70 > pLDDT > 50)')
print('Light blue: High (90 > pLDDT > 70)')
print('Dark blue: Very high (pLDDT > 90)')
print('The visualisation is interactive - zoom in and look where your mutations are.')

#Highlight
with open(OUTDIR+'/AF-'+ID+'-F1-model_v4.pdb',"r") as ifile:
    system = "".join([x for x in ifile])
i = 0
for line in system.split("\n"):
    split = line.split()
    if len(split) == 0 or split[0] != "ATOM":
        continue
    idx = int(line[22:26])
    plddt = float(line[61:66])

    if plddt<50:
      color = "orange"
      view.setStyle({'model': -1, 'serial': i+1}, {"cartoon": {'color': color}})
    if (plddt>=50 and plddt<70):
      color = "yellow"
      view.setStyle({'model': -1, 'serial': i+1}, {"cartoon": {'color': color}})
    if (plddt>=70 and plddt<90):
      color = "deepskyblue"
      view.setStyle({'model': -1, 'serial': i+1}, {"cartoon": {'color': color}})
    if plddt>=90:
      color = "blue"
      view.setStyle({'model': -1, 'serial': i+1}, {"cartoon": {'color': color}})

    if idx in TARGET_POSITIONS:
        color = "magenta"
        view.setStyle({'model': -1, 'serial': i+1}, {"sphere": {'color': color}})

    i += 1

view.zoomTo()
view.show()


All results will be saved to /content/Q6FXU7/
Protein of interest downloaded
Viewing the POI Q6FXU7 in cartoon style and the positions for the input mutations in magenta spheres.
The structure is coloured by plDDT (confindence - see next section):
Orange: Very low (pLDDT < 50)
Yellow: Low (70 > pLDDT > 50)
Light blue: High (90 > pLDDT > 70)
Dark blue: Very high (pLDDT > 90)
The visualisation is interactive - zoom in and look where your mutations are.


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [9]:
#@title #Map your mutations to high confidence regions

#@markdown Not all protein regions can be predicted equally well.
#@markdown How well a protein region can be predicted is related to the [coevolution](https://en.wikipedia.org/wiki/Coevolution) in that region - a measure of how important the exact structure of that region is.

#@markdown Here we will map the mutations to high confidence regions
#@markdown - *plDDT* - a measure of structural confidence (default=80)

[669, 1048, 270]